In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
import os
#처음 연결할 경우에만 사용
#path = '/content/gdrive/MyDrive/MachineVision'
#os.mkdir(path)

#cv.imwrite('/content/gdrive/MyDrive/MachineVision/HW4_result/result1_1.jpg',img)
!pip install opencv-contrib-python==4.4.0.44

In [19]:
import numpy as np
import cv2 as cv
from google.colab.patches import cv_imshow

In [33]:
im_src = cv.imread('/content/gdrive/MyDrive/MachineVision/HW5_image/banner.jpg')
size = im_src.shape
pts_src = np.array([[0,0, 1],[size[1]-1,0, 1],[size[1]-1,size[0]-1, 1],[0,size[0]-1, 1]],dtype=float)
im_dst = cv.imread('/content/gdrive/MyDrive/MachineVision/HW5_image/theater.jpg')
pts_dst = np.array([[1091, 1384, 1],[2784, 1402, 1],[2826, 1725, 1],[1065, 1724, 1]], dtype=float)
pts_dst2 = np.array([[1091, 1384],[2784, 1402],[2826, 1725],[1065, 1724]], dtype=float)

In [34]:
A1 = np.hstack((np.zeros((len(pts_dst),3)),
               -np.expand_dims(pts_dst[:,2],axis=1)*pts_src,
               np.expand_dims(pts_dst[:,1], axis =1)*pts_src))

A2 = np.hstack((np.expand_dims(pts_dst[:,2],axis = 1)*pts_src,
               np.zeros((len(pts_dst),3)),
               -np.expand_dims(pts_dst[:,0], axis =1)*pts_src))

A = np.vstack((A1,A2))

In [35]:
u, s, vh = np.linalg.svd(A, full_matrices=True)
h = vh[-1,:]/vh[-1,-1];
h = np.reshape(h,(3,3))

In [23]:
im_temp = cv.warpPerspective(im_src, h, (im_dst.shape[1],im_dst.shape[0]))
cv.fillConvexPoly(im_dst, pts_dst2.astype(int), 0)

cv_imshow(im_dst)
cv_imshow(im_temp)
cv.imwrite('/content/gdrive/MyDrive/MachineVision/HW5_result/result1_1.jpg',im_dst)
cv.imwrite('/content/gdrive/MyDrive/MachineVision/HW5_result/result1_2.jpg',im_temp)
im_dst = im_dst + im_temp

cv_imshow(im_dst)
cv.imwrite('/content/gdrive/MyDrive/MachineVision/HW5_result/result1_3.jpg',im_dst)

Output hidden; open in https://colab.research.google.com to view.

In [24]:
im_src2 = cv.resize(cv.imread('/content/gdrive/MyDrive/MachineVision/HW5_image/sculpture1.jpg'), dsize=(1000,563))
im_dst2 = cv.resize(cv.imread('/content/gdrive/MyDrive/MachineVision/HW5_image/sculpture2.jpg'), dsize=(1000,563))
gray_src = cv.cvtColor(im_src2, cv.COLOR_BGR2GRAY)
gray_dst = cv.cvtColor(im_dst2, cv.COLOR_BGR2GRAY)

In [25]:
sift = cv.SIFT_create ()
kp1,des1 = sift.detectAndCompute(im_src2, None)
kp2, des2 = sift.detectAndCompute(im_dst2, None)

In [26]:
im_src2_blob = cv.drawKeypoints(gray_src,kp1,im_src2, flags = cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv_imshow(im_src2_blob)

im_dst2_blob = cv.drawKeypoints(gray_dst,kp2,im_dst2, flags = cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv_imshow(im_dst2_blob)

Output hidden; open in https://colab.research.google.com to view.

In [27]:
cv.imwrite('/content/gdrive/MyDrive/MachineVision/HW5_result/result2_1.jpg',im_src2_blob)
cv.imwrite('/content/gdrive/MyDrive/MachineVision/HW5_result/result2_2.jpg',im_dst2_blob)
im_src2 = cv.resize(cv.imread('/content/gdrive/MyDrive/MachineVision/HW5_image/sculpture2.jpg'), dsize=(1000,563))
im_dst2 = cv.resize(cv.imread('/content/gdrive/MyDrive/MachineVision/HW5_image/sculpture1.jpg'), dsize=(1000,563))

In [28]:
kp1,des1 = sift.detectAndCompute(im_src2, None)
kp2, des2 = sift.detectAndCompute(im_dst2, None)

In [29]:
bf = cv.BFMatcher()
matches = bf.match(des1,des2)
matches = sorted(matches, key = lambda x:x.distance)
im_match = cv.drawMatches(im_src2, kp1, im_dst2, kp2, matches[:15], None, 
                        flags = cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv_imshow(im_match)
cv.imwrite('/content/gdrive/MyDrive/MachineVision/HW5_result/result2_3.jpg',im_match)

Output hidden; open in https://colab.research.google.com to view.

In [30]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)

best_match = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        best_match.append(m)

In [31]:
if len(best_match)>10:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in best_match]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in best_match]).reshape(-1,1,2)

    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()

else:
    print("Not enough matches are found - %d/%d" % (len(best_match),10) ,matchesMask = None)

In [32]:
im2_result = cv.warpPerspective(im_src2, M, (im_dst2.shape[1] + im_src2.shape[1], im_dst2.shape[0]))
cv_imshow(im2_result)
im2_result[0:im_src2.shape[0], 0:im_src2.shape[1]] = im_dst2
cv_imshow(im2_result)
cv.imwrite('/content/gdrive/MyDrive/MachineVision/HW5_result/result2_4.jpg',im2_result)

Output hidden; open in https://colab.research.google.com to view.